In [3]:
using HTTP

# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if !isdir(data_dir)
    mkdir(data_dir)
end

# ImageNetのclass_indexをダウンロードする
## Kerasで用意されているものを利用
## https://github.com/fchollet/deep-learning-models/blob/master/imagenet_utils.py
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
save_path = joinpath(data_dir, "imagenet_class_index.json")

if !isfile(save_path)
    HTTP.open(:GET, url) do http
        open(save_path, "w") do file
            write(file, http)
        end
    end
end

In [7]:
using PyCall
@pyimport zipfile

# アリとハチの画像データをダウンロードし解凍
## PyTorchのチュートリアルで用意されているものを利用
## https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = joinpath(data_dir, "hymenoptera_data.zip")

if !isfile(save_path)
    HTTP.open(:GET, url) do http
        open(save_path, "w") do file
            write(file, http)
        end
    end

    # ZIPファイルを読み込み
    const zip = zipfile.ZipFile(save_path)
    zip.extractall(data_dir)  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    rm(save_path)
end

In [10]:
# 必要パッケージのimport
@pyimport numpy as np
@pyimport json
@pyimport PIL.Image as Image
@pyimport matplotlib.pyplot as plt

# PyTorch関連パッケージのimport
@pyimport torch
@pyimport torchvision
@pyimport torchvision.models as models
@pyimport torchvision.transforms as transforms

In [11]:
# PyTorchのバージョン確認
println("PyTorch Version: ", torch.__version__)
println("Torchvision Version: ", torchvision.__version__)

PyTorch Version: 1.7.1
Torchvision Version: 0.8.2


In [12]:
# 学習済みのVGG-16モデルをロード
# 初めて実行する際は、学習済みパラメータをダウンロードするため、実行に時間がかかります

# VGG-16モデルのインスタンスを生成
use_pretrained = true  # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)
net.eval()  # 推論モードに設定

# モデルのネットワーク構成を出力
println(net)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/user/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|███████████████████████████████████████| 528M/528M [00:09<00:00, 59.2MB/s]

PyObject VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, d